In [1]:
import numpy as np
import pandas as pd

In [2]:
import os


def eachFile(filepath):#read all the files
    """
    get all the files in the dir
    """
    pathDir =  os.listdir(filepath)
    files = []
    for allDir in pathDir:
        child = os.path.join('%s%s' % (filepath, allDir))
        if '.txt' in child:
            files.append(child)
    return files


def get_text(values, path):#read the text in the file
    """
    get the text value in the file
    """
    text = open(path, 'r', encoding='UTF-8')
    for line in text:
        values.append(line)
        
def get_values(names):
    ret = []
    for name in names:
        suf = name.split('_')[1]
        ret.append(int(suf.split('.')[0]))
    return ret

In [3]:
# get all the text in the dir 
files = eachFile('aclImdbdemo/train/neg/')
neg_v = get_values(files)
neg_files = []
for file in files:
    neg_files.append(file)

files = eachFile('aclImdbdemo/train/pos/')
pos_v = get_values(files)
pos_files = []
for file in files:
    pos_files.append(file)
    
files = eachFile('aclImdbdemo/train/unsup/')
unsup_v = get_values(files)
unsup_files = []
for file in files:
    unsup_files.append(file)
        
neg_text = []
pos_text = []
unsup_text = []
for file in neg_files:
    get_text(neg_text, file)
for file in pos_files:
    get_text(pos_text, file)
for file in unsup_files:
    get_text(unsup_text, file)

In [4]:
# get all the text in the dir 
files = eachFile('aclImdbdemo/test/neg/')
negt_v = get_values(files)
negt_files = []
for file in files:
    negt_files.append(file)

files = eachFile('aclImdbdemo/test/pos/')
post_v = get_values(files)
post_files = []
for file in files:
    post_files.append(file)
    
        
neg_test = []
pos_test = []
for file in negt_files:
    get_text(neg_test, file)
for file in post_files:
    get_text(pos_test, file)

In [5]:
import nltk
# count all the words in the text
import re,collections  
def get_words(file):   
    words_box=[]  
    for line in file:
        line = line.lower()
        tokens = nltk.word_tokenize(line)
        words_box.extend(tokens)                 
    return collections.Counter(words_box) 

In [6]:
unsup_text = []
for file in unsup_files:
    get_text(unsup_text, file)

In [7]:
word_count =  get_words(neg_text)#Create a dictionary
word_count = dict(word_count, ** get_words(pos_text))
word_count = dict(word_count, ** get_words(unsup_text))
word_count = dict(word_count, ** get_words(neg_test))#Create a dictionary
word_count = dict(word_count, ** get_words(pos_test))


In [8]:
print(len(neg_test))

2500


In [9]:
symbol='??,???,????,?????,??????,???????,????????,?????????,??????????,???????????,????????????,???????????????,!!,!!!,!!!!,!!!!!,!!!!!!,!!!!!!!,!!!!!!!!,!!!!!!!!!,!!!!!!!!!!,!!!!!!!!!!!,!!!!!!!!!!!!,?!!!!!!!!!!!!!!,!?,?!,!??,?!?,??!,!??!,((((((((((((((((((,xx,xxx,xxxx,xxxxx,:),:(,:D,:X,:x,:C,:c,:P,:p,:>,:<,:[,:],:|,:#,:o,:O,:/,:\,;),;(,;D,;X,;x,;C,;c,;P,;p,;>,;<,;[,;],;|,;#,;o,;O,;/,;\,;-),;-(,;-D,;-X,;-x,;-C,;-c,;-P,;-p,;->,;-<,;-[,;-],;-|,;-#,;-o,;-O,;-/,;-\,:-),:-(,:-D,:-X,:-x,:-C,:-c,:-P,:-p,:->,:-<,:-[,:-],:-|,:-#,:-o,:-O,:-/,:-\,:\'(,((.,;_;,:-,???-??,!),(:,:**-(,XD,D;,):'
symbols = symbol.split(',')

In [10]:
# get the category of the word by using nltk method
from nltk.corpus import stopwords
import re
tags = set(['CC','DT','MD','IN','NN', 'NNP', 'NNS', 'NNPS','UH', 'VB','VBD', 'VBG','MD', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS'])
def filter(text):
    # drop all the stop word and the word only show 1 time
#     words = []
    text = text.lower()
#     pat_letter = re.compile(r'[^a-zA-Z \']+')
#     text = pat_letter.sub(' ', text).strip().lower()
#     for w in text.split():
#         if w in stopwords.words('english'):
#             continue
#         if w in symbols:
#             words.append(w)
#             continue
#         if w is not None and w in word_count and word_count[w] > 1:
#             words.append(w)
            
    words = [w for w in text.split() if((w in word_count and word_count[w] > 1) or w in symbols)]
    # get the categoyr of word
    pos_tags =nltk.pos_tag(words)
    ret = []
    #filter
    for word,pos in pos_tags:
        if (pos in tags):
            ret.append(word)
    return ' '.join(ret)

In [11]:
#get the text after filtering
neg_text = list(map(filter, neg_text))
pos_text = list(map(filter, pos_text))
unsup_text = list(map(filter, unsup_text))
neg_test =  list(map(filter, neg_test))
pos_test = list(map(filter,pos_test))

In [12]:
import pandas as pd
import numpy as np
# structer the data
neg_pd = pd.DataFrame(columns = ['content', 'label'])
pos_pd = pd.DataFrame(columns = ['content', 'label'])
unsup_pd = pd.DataFrame(columns = ['content', 'label'])
negt_pd = pd.DataFrame(columns = ['content', 'label'])
post_pd = pd.DataFrame(columns = ['content', 'label'])
neg_pd['content'] = neg_text
neg_pd['label'] = neg_v
pos_pd['content'] = pos_text
pos_pd['label'] = pos_v
unsup_pd['content'] = unsup_text
unsup_pd['label'] = unsup_v
negt_pd['content'] = neg_test
negt_pd['label'] = negt_v
post_pd['content'] = pos_test
post_pd['label'] = post_v
data = pd.concat([pos_pd, neg_pd, unsup_pd, negt_pd, post_pd], axis = 0, ignore_index = True)

In [13]:
data.label.value_counts()

0     10000
10     2007
1      1950
4      1128
8      1121
3      1005
9       978
2       917
7       894
Name: label, dtype: int64

In [14]:
data.loc[data.label == 1, 'label'] = 2
data.loc[data.label == 3, 'label'] = 4
data.loc[data.label == 5, 'label'] = 6
data.loc[data.label == 7, 'label'] = 8
data.loc[data.label == 9, 'label'] = 10

In [15]:
data.head(100)
data.to_csv("scoreallafterfilter.csv",index=False,sep=',')

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
count_vec = CountVectorizer()
# cross validation
x_train, x_test, y_train, y_test = train_test_split(data.content, data.label, test_size=0.1, random_state=23)
# word count by CountVectorizer
x_train_mnb = count_vec.fit_transform(x_train)
x_test_mnb = count_vec.transform(x_test)

In [17]:
from sklearn import svm  
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.externals import joblib
#predict by SVM
dtc = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)

dtc.fit(x_train_mnb, y_train)
print(classification_report(y_test, dtc.predict(x_test_mnb)))

G:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


             precision    recall  f1-score   support

          0       0.68      0.55      0.61       978
          2       0.47      0.52      0.49       289
          4       0.24      0.50      0.33       204
          8       0.22      0.24      0.23       199
         10       0.48      0.35      0.40       330

avg / total       0.52      0.48      0.49      2000

